# 5. LangChain解説

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [ ]:
# 本書で動作確認済みのchromadb v0.4.10が動作するよう、pydanticを動作確認済みのv1.10.13に固定します。
# この処理はpydanticが最初に使われる前に実行する必要があるため、ノートブックの先頭のあたりで実行してください。
# もしもうまく動作しない場合、Google Colabのランタイムを再起動または削除して先頭から実行し直してください。
#
# 関連: https://github.com/yoshidashingo/langchain-book/issues/16
!pip install pydantic==1.10.13

In [ ]:
!pip install langchain==0.0.292 openai==0.28.0

## 5-1 Data connection

### Document loaders

In [ ]:
!pip install GitPython==3.1.36

In [ ]:
from langchain.document_loaders import GitLoader

def file_filter(file_path):
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

### Document transformers

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

## Text embedding models

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
!pip install tiktoken==0.5.1

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

### Vector stores

In [ ]:
!pip install chromadb==0.4.10

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

### Retriever

In [ ]:
retriever = db.as_retriever()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

context_docs = retriever.get_relevant_documents(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

### RetrievalQA（Chain）

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=retriever)

result = qa_chain.run(query)
print(result)

## 5-2 Agents

### Agentsの使用例

In [ ]:
import langchain

langchain.verbose = True

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"])
agent_chain = initialize_agent(
    tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

result = agent_chain.run("sample_dataディレクトリにあるファイルの一覧を教えて")
print(result)

### Tools

In [ ]:
from langchain.tools import Tool

def my_super_func(param):
    return "42"

tools = [
    Tool.from_function(
        func=my_super_func,
        name="The_Answer",
        description="生命、宇宙、そして万物についての究極の疑問の答え"
    ),
]

In [ ]:
agent_chain = initialize_agent(
    tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

result = agent_chain.run("この世界の真理を教えてください")
print(result)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain

summarize_template = """以下の文章を結論だけ一言に要約してください。

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

tools = [
    Tool.from_function(
        func=summarize_chain.run,
        name="Summarizer",
        description="Text summarizer"
    ),
]

In [ ]:
agent_chain = initialize_agent(
    tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

text = """以下を要約してください。

こんにちは！私はChatGPTと呼ばれるAI言語モデルです。OpenAIが開発したGPT-3.5アーキテクチャに基づいています。私は自然言語理解と生成に特化しており、さまざまなトピックに関する質問に答えたり、おしゃべりしたりすることが得意です。
私のトレーニングデータは2021年9月までの情報に基づいているため、それ以降の出来事については知識がありません。ですが、できる限りお手伝いすることに努めます。
質問や会話、情報の共有など、どんなお手伝いでもお気軽にお申し付けください！よろしくお願いします。"""

result = agent_chain.run(text)
print(result)

### Function callingを使うOpenAI Functions Agent

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"])
agent_chain = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS)

result = agent_chain.run("sample_dataディレクトリにあるファイルの一覧を教えて")
print(result)

### 複数のツールを一度に使うOpenAI Multi Functions Agent

In [ ]:
!pip install duckduckgo-search==4.1.0

In [ ]:
import langchain

langchain.debug = True
langchain.verbose = False

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["ddg-search"])
agent_chain = initialize_agent(tools, chat, agent=AgentType.OPENAI_MULTI_FUNCTIONS)

result = agent_chain.run("東京と大阪の天気を教えて")
print(result)

### （コラム）Function callingを応用したOurputParser・Extraction・Tagging

In [ ]:
import json

from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}
text = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_extraction_chain(schema, chat)

people = chain.run(text)
print(json.dumps(people, indent=2))

## まとめ

### （コラム）Evaluation

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import load_evaluator

chat = ChatOpenAI(model="gpt-4", temperature=0)

evaluator = load_evaluator("qa", eval_llm=chat)

result = evaluator.evaluate_strings(
    input="私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？",
    prediction="""1最初に10個のリンゴを買い、その中から隣人と修理工にそれぞれ2個ずつ渡しました。そのため、まず手元に残ったリンゴは10 - 2 - 2 = 6個となります。

その後、さらに5個のリンゴを買い、1つ食べました。これにより手元のリンゴは6 + 5 - 1 = 10個となります。""",
    reference="10個",
)

print(result)

上記の入力は [Chain-of-Thoughtプロンプティング | Prompt Engineering Guide](https://www.promptingguide.ai/jp/techniques/cot) から引用しました。